In [ ]:
!pip uninstall h5py
!pip install h5py==2.9

Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/h5py-3.1.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/h5py.libs/libaec-9c9e97eb.so.0.0.10
    /usr/local/lib/python3.8/dist-packages/h5py.libs/libhdf5-00e8fae8.so.200.0.0
    /usr/local/lib/python3.8/dist-packages/h5py.libs/libhdf5_hl-383c339f.so.200.0.0
    /usr/local/lib/python3.8/dist-packages/h5py.libs/libsz-e7aa62f5.so.2.0.1
    /usr/local/lib/python3.8/dist-packages/h5py.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.8/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-3.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 14.6 MB/s 


In [ ]:
%%bash

conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
pip install ftfy regex tqdm
pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-y61yf3j6
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369408 sha256=c015ae4b322de3d7b12157ab0078f888dfbf25742e2d33eb473a197c370c1d32
  Stored in directory: /tmp/pip-ephem-wheel-cache-fsgcuk95/wheels/ab/4f/3a/5e51521b55997aa6f0690e095c08824219753128ce8d9969a3
Successfully built clip


bash: line 2: conda: command not found
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-y61yf3j6


In [ ]:
import os
import cv2
import numpy as np
import sys
import glob
import json
import h5py

from tqdm import tqdm
import torch
import torchvision.transforms as transforms
import torchvision.models as models

root = os.sep.join(os.getcwd().split(os.sep)[:-4])

In [ ]:
import clip
model_type = "ViT-B/32"
device = torch.device('cuda:0')
clip_model, preprocess = clip.load(model_type, device=device, jit=False)
from PIL import Image

print(device)

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 113MiB/s]


cuda:0


In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/'My Drive'/CV_Project/

Mounted at /content/drive
/content/drive/My Drive/CV_Project


In [ ]:
# cite from : https://github.com/nasib-ullah/video_feature_extraction/blob/main/Spatio_temporal_feature_extraction/ConvLSTM_feature_extraction.ipynb
# data_transform = transforms.Compose([transforms.ToPILImage(),transforms.Resize((299,299)),transforms.ToTensor()])
def video2tensor(video_path):
    vidObj = cv2.VideoCapture(video_path) 
    count = 0
    success = 1
    frames = []
    fail = 0
    while success: 
        # OpenCV Uses BGR Colormap
        success, image = vidObj.read() 
        if success:
            RGBimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #transform images
            #print(RGBimage.shape)
            RGBimage = preprocess(Image.fromarray(RGBimage)).to(device)
            frames.append(RGBimage)
            count += 1
        else:
            fail += 1
    vidObj.release()
    if(frames):
      frames = torch.stack(frames)
      #frames = torch.from_numpy(frames)
      # take 28 frames per clip uniformly
      interval = count//28
      frames = frames[range(0,interval*28,interval)]
      return frames,count,fail
    return {},count,fail

# MSRVTT

In [ ]:
dset_path = os.path.join(root,'dataset')
msrvtt_trnval_path = os.path.join(dset_path,'TrainValVideo')
msrvtt_trnval_name_list = glob.glob(msrvtt_trnval_path+os.sep+'*')

# msrvtt_test_path = os.path.join(dset_path,'TestVideo')
# msrvtt_test_name_list = glob.glob(msrvtt_test_path+os.sep+'*')
# msrvtt_name_list = msrvtt_trnval_name_list+msrvtt_test_name_list

In [ ]:
with torch.no_grad():
    with h5py.File('MSRVTT_TRNVAL_CLIP_FEATURES.hdf5', 'w') as f:
        for name in tqdm(msrvtt_trnval_name_list):
            ide = name.split(os.sep)[-1].split('.')[0]
            if not f.__contains__(ide):
                tensor,_,_ = video2tensor(name)
                if(len(tensor)!=0):
                    with torch.no_grad():
                        output_features = clip_model.encode_image(tensor.to(device)).cpu()
                    f.create_dataset(ide, data = output_features.cpu().numpy())

100%|██████████| 7010/7010 [3:37:51<00:00,  1.86s/it]


In [ ]:
f = h5py.File('MSRVTT_TRNVAL_CLIP_FEATURES.hdf5','r+')
print(len(f))

7009


In [ ]:
# path = 'dd.mp4'
# with torch.no_grad():
#     with h5py.File('test.hdf5', 'w') as f:
#     # f = h5py.File('test.hdf5','r+')
#       ide = path.split(os.sep)[-1].split('.')[0]
#       if not f.__contains__(ide):
#         tensor,_,_ = video2tensor(path)
#         if(len(tensor)!=0):
#           with torch.no_grad():
#               output_features = clip_model.encode_image(tensor.to(device)).cpu()
#           f.create_dataset(ide, data = output_features.cpu().numpy())

In [ ]:
# f = h5py.File('test.hdf5','r')
# for k in f.keys():
#   print(f[k].value) 